In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd 

from torchvision import datasets, transforms
import numpy as np
import random

from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score

In [2]:
#normal transformationto every image in trainset
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#transformation to images that belong to minor class 
transform_train_minor = transforms.Compose([
    transforms.RandomPerspective(),
    transforms.ColorJitter(0.1,0.2,0.2,0.1),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
])

#normal transformationto every image in testset
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [11]:
#loading train dataset
dataset_train = datasets.CIFAR10('../data/cifar', train=True, download=True, transform=transform_train)
#loading test dataset
testset = datasets.CIFAR10('../data/cifar-t',train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
#this function is sampling the cifar image index in non iid way, each client will have majority of 2 class and 8 minor class
def cifar_noniid2(dataset, num_users, p):

    idxs = np.arange(len(dataset),dtype=int)
    labels = np.array(dataset.targets)
    label_list = np.unique(dataset.targets)
    
    # sort labels
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    #print(idxs_labels)
    idxs = idxs_labels[0,:]
    idxs = idxs.astype(int)
    n_data=1000
    dict_users = {i: np.array([], dtype='int64') for i in range(num_users)}

    #Sample majority class for each user
    user_majority_labels = []
    for i in range(num_users):
        majority_labels = np.random.choice(label_list, 2, replace = False)
        user_majority_labels.append(majority_labels)

        #label_list = list(set(label_list) - set(majority_labels))
       # print(i,majority_labels)
        majority_label_idxs = (majority_labels[0] == labels[idxs]) | (majority_labels[1] == labels[idxs])        
        
        sub_data_idxs = np.random.choice(idxs[majority_label_idxs], int(p*n_data), replace = False)
        
        dict_users[i] = np.concatenate((dict_users[i],sub_data_idxs))
        idxs = np.array(list(set(idxs) - set(sub_data_idxs)))
        
        #assigning minor classes to each client
    if(p < 1.0):
        for i in range(num_users):
            majority_labels = user_majority_labels[i]
            
            non_majority_label_idxs = (majority_labels[0] != labels[idxs]) & (majority_labels[1] != labels[idxs])
            
            sub_data_idxs = np.random.choice(idxs[non_majority_label_idxs], int((1-p)*n_data), replace = False)
            
            dict_users[i] = np.concatenate((dict_users[i], sub_data_idxs))
            idxs = np.array(list(set(idxs) - set(sub_data_idxs)))

    return dict_users

In [13]:
dict_users = cifar_noniid2(dataset_train,49,0.4)

In [14]:
len(dict_users[0])

1000

In [15]:
dataset_train=list(dataset_train)
count=50000 #last index of cifar train img, so we have to start from that

#using image augmentation twice 2 and add 2 transformed image 
for j in range(5):
    for i in dict_users[j][400:]: #starting from 400 because after 400 array will conatin indexes of minor class image
        new_image=transform_train_minor(dataset_train[i][0])
        label=dataset_train[i][1]
        dataset_train.append([new_image,label])   
        count += 1
        new_tranform_idxs=count
        dict_users[j] = np.append(dict_users[j], new_tranform_idxs)  
        
for j in range(5):
    for i in dict_users[j][400:1000]:
        new_image=transform_train_minor(dataset_train[i][0])
        label=dataset_train[i][1]
        dataset_train.append([new_image,label])   
        count += 1
        new_tranform_idxs=count
        dict_users[j] = np.append(dict_users[j], new_tranform_idxs)  

C:\Users\ujjaw\Anaconda3\lib\site-packages\torchvision\transforms\functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")


In [16]:
# code to check number of images/classes
temp=list()
for i in dict_users[0]:
     temp.append(dataset_train[i][1])

for i in range(10):
     print("class",i,'->',temp.count(i))

class 0 -> 276
class 1 -> 153
class 2 -> 224
class 3 -> 206
class 4 -> 261
class 5 -> 213
class 6 -> 216
class 7 -> 211
class 8 -> 246
class 9 -> 194


In [18]:
#creating a batch data of 2500 imgs per client
lb=LabelBinarizer()
def batch_data(data_shard, bs=2500):
    label=[]
    
    for i in range(len(data_shard)):
        label.append(dataset_train[i][1])
    label=lb.fit_transform(label)
    
    data=[]
    for i in range(len(data_shard)):
        data.append(dataset_train1[i]) 
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [19]:
#changing the tensor datatype to numpy datatype
dataset_train1=[]
for i in range(len(dataset_train)):
    temp=dataset_train[i][0].numpy()
    dataset_train1.append(np.rollaxis(temp,0,3))

#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in dict_users.items():
    clients_batched[client_name] = batch_data(data)

In [20]:
testset1=[]
for i in range(len(testset)):
    temp=testset[i][0].numpy()
    testset1.append(np.rollaxis(temp,0,3))
    
#process and batch the test set  
label=[]
for i in range(2000): #getting 2000 images only from testset to test the model
    label.append(testset[i][1])
label=lb.fit_transform(label)

data=[]
for i in range(2000):
    data.append(testset1[i])   
test_batched = tf.data.Dataset.from_tensor_slices((list(data), list(label))).batch(1000)

In [21]:
#building a simple NN
class SimpleMLP:
    @staticmethod
    def build(classes):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32,32,3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(classes, activation='softmax'))
        return model


In [24]:
#defining parameters
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )    

In [23]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [27]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(10)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    #print(global_weights)
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=10, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

KeyboardInterrupt: 